# Task
Clean the `corporate_rating.csv` dataset by inspecting it, merging rating classes, encoding target labels, handling missing values, and saving the cleaned data to "credit_ratings_tabular_clean.csv".

## Load data

### Subtask:
Load the `corporate_rating.csv` file into a pandas DataFrame.


**Reasoning**:
Import pandas and load the CSV file into a DataFrame.



In [1]:
import pandas as pd

df = pd.read_csv('corporate_rating.csv')

In [2]:
df.head()

,Rating,Name,Symbol,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,...,effectiveTaxRate,freeCashFlowOperatingCashFlowRatio,freeCashFlowPerShare,cashPerShare,companyEquityMultiplier,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover
0,A,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,11/27/2015,Consumer Durables,0.945894,0.426395,0.099690,44.203245,...,0.202716,0.437551,6.810673,9.809403,4.008012,0.049351,7.057088,15.565438,0.058638,3.906655
1,BBB,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,2/13/2014,Consumer Durables,1.033559,0.498234,0.203120,38.991156,...,0.074155,0.541997,8.625473,17.402270,3.156783,0.048857,6.460618,15.914250,0.067239,4.002846
2,BBB,Whirlpool Corporation,WHR,Fitch Ratings,3/6/2015,Consumer Durables,0.963703,0.451505,0.122099,50.841385,...,0.214529,0.513185,9.693487,13.103448,4.094575,0.044334,10.491970,18.888889,0.074426,3.483510
3,BBB,Whirlpool Corporation,WHR,Fitch Ratings,6/15/2012,Consumer Durables,1.019851,0.510402,0.176116,41.161738,...,1.816667,-0.147170,-1.015625,14.440104,3.630950,-0.012858,4.080741,6.901042,0.028394,4.581150
4,BBB,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,10/24/2016,Consumer Durables,0.957844,0.495432,0.141608,47.761126,...,0.166966,0.451372,7.135348,14.257556,4.012780,0.053770,8.293505,15.808147,0.058065,3.857790


In [3]:
df.rename(columns={'Symbol': 'Ticker'}, inplace=True)


In [4]:
total_ticker=df["Ticker"].unique()
len(total_ticker)

593

## Initial inspection

### Subtask:
Inspect column names, datatypes, missingness, and distributions.


**Reasoning**:
Inspect the DataFrame by printing column names and their data types, calculating missing value percentages, generating descriptive statistics for numerical columns, and examining the distribution of the 'Rating' column to understand the data structure, identify potential issues like missing data, and understand the distribution of the target variable.



In [5]:
print(df.info())
print("\nPercentage of missing values:\n", df.isnull().sum() * 100 / len(df))
print("\nDescriptive statistics for numerical columns:\n", df.describe())
print("\nDistribution of the 'Rating' column:\n", df['Rating'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Rating                              2029 non-null   object 
 1   Name                                2029 non-null   object 
 2   Ticker                              2029 non-null   object 
 3   Rating Agency Name                  2029 non-null   object 
 4   Date                                2029 non-null   object 
 5   Sector                              2029 non-null   object 
 6   currentRatio                        2029 non-null   float64
 7   quickRatio                          2029 non-null   float64
 8   cashRatio                           2029 non-null   float64
 9   daysOfSalesOutstanding              2029 non-null   float64
 10  netProfitMargin                     2029 non-null   float64
 11  pretaxProfitMargin                  2029 no

## Merge rating classes

### Subtask:
Map 'AAA' to 'AA+' and 'CC', 'C', 'D' to 'CCC–' to create 6 final classes.


**Reasoning**:
Create a dictionary to map the original rating classes to the new 6 classes, apply this mapping to the 'Rating' column to create a new column 'Rating_Merged', and verify the distribution of the new column.



In [6]:
rating_mapping = {
    'AAA': 'AA+',
    'CC': 'CCC–',
    'C': 'CCC–',
    'D': 'CCC–'
}

df['Rating_Merged'] = df['Rating'].replace(rating_mapping)

print(df['Rating_Merged'].value_counts())

Rating_Merged
BBB     671
BB      490
A       398
B       302
AA       89
CCC      64
CCC–      8
AA+       7
Name: count, dtype: int64


## Encode target labels

### Subtask:
Create both multiclass and binary (Investment Grade) encoded target labels.


**Reasoning**:
Encode the 'Rating_Merged' column into both multiclass and binary labels, then display the value counts.



In [7]:
from sklearn.preprocessing import LabelEncoder

# Multiclass encoding
label_encoder = LabelEncoder()
df['Rating_Encoded_Multiclass'] = label_encoder.fit_transform(df['Rating_Merged'])

# Binary encoding (Investment Grade vs. Not Investment Grade)
investment_grade_ratings = ['AAA', 'AA+', 'AA', 'A+', 'A', 'A-', 'BBB+', 'BBB', 'BBB-']
df['Rating_Encoded_Binary'] = df['Rating_Merged'].apply(lambda x: 1 if x in investment_grade_ratings else 0)

# Display value counts for verification
print("Value counts for Multiclass Encoded Ratings:\n", df['Rating_Encoded_Multiclass'].value_counts())
print("\nValue counts for Binary Encoded Ratings:\n", df['Rating_Encoded_Binary'].value_counts())

Value counts for Multiclass Encoded Ratings:
 Rating_Encoded_Multiclass
5    671
4    490
0    398
3    302
1     89
6     64
7      8
2      7
Name: count, dtype: int64

Value counts for Binary Encoded Ratings:
 Rating_Encoded_Binary
1    1165
0     864
Name: count, dtype: int64


## Handle missing values

### Subtask:
Implement a strategy for handling missing values, potentially using median imputation for ratios and KNN or iterative imputation for other features, while adding indicator columns for original missingness.


**Reasoning**:
Identify columns with missing values and their types, then create indicator columns for missing values.



In [8]:
# Check for missing values
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]

print("Columns with missing values:\n", missing_values)

# Create indicator columns for original missingness
for col in missing_values.index:
    df[col + '_missing_indicator'] = df[col].isnull().astype(int)

display(df.head())

Columns with missing values:
 Series([], dtype: int64)


,Rating,Name,Ticker,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,...,cashPerShare,companyEquityMultiplier,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover,Rating_Merged,Rating_Encoded_Multiclass,Rating_Encoded_Binary
0,A,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,11/27/2015,Consumer Durables,0.945894,0.426395,0.099690,44.203245,...,9.809403,4.008012,0.049351,7.057088,15.565438,0.058638,3.906655,A,0,1
1,BBB,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,2/13/2014,Consumer Durables,1.033559,0.498234,0.203120,38.991156,...,17.402270,3.156783,0.048857,6.460618,15.914250,0.067239,4.002846,BBB,5,1
2,BBB,Whirlpool Corporation,WHR,Fitch Ratings,3/6/2015,Consumer Durables,0.963703,0.451505,0.122099,50.841385,...,13.103448,4.094575,0.044334,10.491970,18.888889,0.074426,3.483510,BBB,5,1
3,BBB,Whirlpool Corporation,WHR,Fitch Ratings,6/15/2012,Consumer Durables,1.019851,0.510402,0.176116,41.161738,...,14.440104,3.630950,-0.012858,4.080741,6.901042,0.028394,4.581150,BBB,5,1
4,BBB,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,10/24/2016,Consumer Durables,0.957844,0.495432,0.141608,47.761126,...,14.257556,4.012780,0.053770,8.293505,15.808147,0.058065,3.857790,BBB,5,1


## Save cleaned data

### Subtask:
Save the cleaned tabular data to a new CSV file named `credit_ratings_tabular_clean.csv`.


**Reasoning**:
Save the cleaned DataFrame to a CSV file without the index.



In [9]:
df.to_csv('01_credit_ratings_tabular_clean.csv', index=False)

## Summary:

### Data Analysis Key Findings

*   The initial dataset contained 2029 entries and 31 columns, with no missing values.
*   The `Rating` column was successfully processed by merging 'AAA' to 'AA+' and 'CC', 'C', 'D' to 'CCC–', resulting in a new column `Rating_Merged` with 8 distinct classes.
*   Two new target label columns were created: `Rating_Encoded_Multiclass` (8 classes) and `Rating_Encoded_Binary` (Investment Grade vs. Not Investment Grade).

### Insights or Next Steps

*   The absence of missing values simplifies the data preparation process, allowing for direct use in modeling.
*   Further analysis should investigate the distribution of the merged and encoded rating classes, particularly the low counts in some categories, which might impact model training.


## reating data as year quater

In [10]:
# Ensure rating_date is a datetime
df['rating_date'] = pd.to_datetime(df['Date'], errors='coerce')

# Define a helper function to map to fiscal quarter
def get_fiscal_quarter(date):
    """Return fiscal year and quarter string (e.g., 2024_Q1) for given date."""
    year = date.year
    qtr = (date.month - 1) // 3 + 1
    return f"{year}_Q{qtr}"

df['year_qtr'] = df['rating_date'].apply(get_fiscal_quarter)

In [11]:
df.head()

,Rating,Name,Ticker,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,...,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover,Rating_Merged,Rating_Encoded_Multiclass,Rating_Encoded_Binary,rating_date,year_qtr
0,A,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,11/27/2015,Consumer Durables,0.945894,0.426395,0.099690,44.203245,...,0.049351,7.057088,15.565438,0.058638,3.906655,A,0,1,2015-11-27,2015_Q4
1,BBB,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,2/13/2014,Consumer Durables,1.033559,0.498234,0.203120,38.991156,...,0.048857,6.460618,15.914250,0.067239,4.002846,BBB,5,1,2014-02-13,2014_Q1
2,BBB,Whirlpool Corporation,WHR,Fitch Ratings,3/6/2015,Consumer Durables,0.963703,0.451505,0.122099,50.841385,...,0.044334,10.491970,18.888889,0.074426,3.483510,BBB,5,1,2015-03-06,2015_Q1
3,BBB,Whirlpool Corporation,WHR,Fitch Ratings,6/15/2012,Consumer Durables,1.019851,0.510402,0.176116,41.161738,...,-0.012858,4.080741,6.901042,0.028394,4.581150,BBB,5,1,2012-06-15,2012_Q2
4,BBB,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,10/24/2016,Consumer Durables,0.957844,0.495432,0.141608,47.761126,...,0.053770,8.293505,15.808147,0.058065,3.857790,BBB,5,1,2016-10-24,2016_Q4


In [12]:
df["Sector"].unique()

array(['Consumer Durables', 'Energy', 'Capital Goods',
       'Consumer Non-Durables', 'Public Utilities', 'Technology',
       'Health Care', 'Consumer Services', 'Miscellaneous',
       'Basic Industries', 'Finance', 'Transportation'], dtype=object)

## now extract data from HTML files

In [13]:
!pip install beautifulsoup4


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
# from bs4 import BeautifulSoup, NavigableString
# import re
# import os

# # Function to extract MD&A from HTML file
# def extract_mda_from_html(file_path):
#     """
#     Extract Management Discussion & Analysis (MD&A) section from an HTML file.
#     Returns the extracted text or None if extraction fails.
#     """
#     try:
#         with open(file_path, "r", encoding="utf-8") as f:
#             html = f.read()
#     except Exception as e:
#         print(f"Error reading file {file_path}: {e}")
#         return None
    
#     soup = BeautifulSoup(html, "html.parser")
    
#     # Regex patterns for start and end
#     pattern_start = re.compile(
#         r"management['']s discussion and analysis", re.IGNORECASE
#     )
    
#     pattern_end = re.compile(
#         r"quantitative.*qualitative", re.IGNORECASE
#     )
    
#     start_href = None
#     end_href = None
    
#     # Find start and end hrefs
#     for a in soup.find_all("a"):
#         text = a.get_text(" ", strip=True)
#         if pattern_start.search(text):
#             start_href = a.get("href")
#             break
    
#     for a in soup.find_all("a"):
#         text = a.get_text(" ", strip=True)
#         if pattern_end.search(text):
#             end_href = a.get("href")
#             break
    
#     if not start_href or not end_href:
#         return None
    
#     # Function to find anchor target
#     def find_anchor_target(href):
#         if not href:
#             return None
        
#         if href.startswith("#"):
#             anchor_name = href[1:]
            
#             # Try ID first
#             t = soup.find(id=anchor_name)
#             if t:
#                 return t
            
#             # Try <a name="">
#             t = soup.find("a", attrs={"name": anchor_name})
#             if t:
#                 return t
            
#             # Try any tag with name=""
#             t = soup.find(attrs={"name": anchor_name})
#             if t:
#                 return t
        
#         return None
    
#     start_anchor = find_anchor_target(start_href)
#     end_anchor = find_anchor_target(end_href)
    
#     if not start_anchor or not end_anchor:
#         return None
    
#     # Extract text between anchors
#     texts = []
    
#     for elem in start_anchor.next_elements:
#         if elem == end_anchor:
#             break
        
#         if isinstance(elem, NavigableString):
#             t = str(elem).strip()
#             if t:
#                 texts.append(t)
    
#     extracted_text = " ".join(texts)
    
#     return extracted_text if extracted_text else None


# # Function to get HTML file path based on Ticker and year_qtr
# def get_html_file_path(ticker, year_qtr):
#     """
#     Construct the path to the HTML file based on Ticker and year_qtr.
#     Example: ticker='HON', year_qtr='2016_Q4' -> 'sec_filings/HON/2016_Q4.html'
#     """
#     base_path = os.path.join("sec_filings", ticker, f"{year_qtr}.html")
#     return base_path


# # Extract MD&A for each row
# print("Extracting MD&A from HTML files...")

# mda_list = []
# success_count = 0
# failed_count = 0

# for idx, row in df.iterrows():
#     ticker = row['Ticker']
#     year_qtr = row['year_qtr']
    
#     html_path = get_html_file_path(ticker, year_qtr)
    
#     if os.path.exists(html_path):
#         print(f"Processing {idx}: {ticker} - {year_qtr}...", end=" ")
#         mda_text = extract_mda_from_html(html_path)
#         if mda_text:
#             mda_list.append(mda_text)
#             success_count += 1
#             print("SUCCESS")
#         else:
#             mda_list.append(None)
#             failed_count += 1
#             print("FAILED")
#     else:
#         print(f"Skipping {idx}: File not found - {html_path}")
#         mda_list.append(None)
#         failed_count += 1

# # Add MD&A column to dataframe
# df['md&a'] = mda_list

# print(f"\nExtraction complete!")
# print(f"Success: {success_count}, Failed/Not Found: {failed_count}")
# print(f"\nMD&A column created. Sample:")
# print(df[['Ticker', 'year_qtr', 'md&a']].head())

# # Check how many rows have MD&A data
# print(f"\nRows with MD&A data: {df['md&a'].notna().sum()}")
# print(f"Rows without MD&A data: {df['md&a'].isna().sum()}")

In [15]:
df.head()

,Rating,Name,Ticker,Rating Agency Name,Date,Sector,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,...,ebitPerRevenue,enterpriseValueMultiple,operatingCashFlowPerShare,operatingCashFlowSalesRatio,payablesTurnover,Rating_Merged,Rating_Encoded_Multiclass,Rating_Encoded_Binary,rating_date,year_qtr
0,A,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,11/27/2015,Consumer Durables,0.945894,0.426395,0.099690,44.203245,...,0.049351,7.057088,15.565438,0.058638,3.906655,A,0,1,2015-11-27,2015_Q4
1,BBB,Whirlpool Corporation,WHR,Egan-Jones Ratings Company,2/13/2014,Consumer Durables,1.033559,0.498234,0.203120,38.991156,...,0.048857,6.460618,15.914250,0.067239,4.002846,BBB,5,1,2014-02-13,2014_Q1
2,BBB,Whirlpool Corporation,WHR,Fitch Ratings,3/6/2015,Consumer Durables,0.963703,0.451505,0.122099,50.841385,...,0.044334,10.491970,18.888889,0.074426,3.483510,BBB,5,1,2015-03-06,2015_Q1
3,BBB,Whirlpool Corporation,WHR,Fitch Ratings,6/15/2012,Consumer Durables,1.019851,0.510402,0.176116,41.161738,...,-0.012858,4.080741,6.901042,0.028394,4.581150,BBB,5,1,2012-06-15,2012_Q2
4,BBB,Whirlpool Corporation,WHR,Standard & Poor's Ratings Services,10/24/2016,Consumer Durables,0.957844,0.495432,0.141608,47.761126,...,0.053770,8.293505,15.808147,0.058065,3.857790,BBB,5,1,2016-10-24,2016_Q4


In [16]:
from bs4 import BeautifulSoup, NavigableString
import re
import os

# Improved MD&A extraction function with multiple strategies
def extract_mda_from_html(file_path, verbose=False):
    """
    Extract Management Discussion & Analysis (MD&A) section from an HTML file.
    Uses multiple strategies to find the MD&A section.
    """
    try:
        with open(file_path, "r", encoding="utf-8", errors='ignore') as f:
            html = f.read()
    except Exception as e:
        if verbose:
            print(f"Error reading file {file_path}: {e}")
        return None
    
    soup = BeautifulSoup(html, "html.parser")
    
    # Strategy 1: Look for MD&A in table of contents links
    mda_text = extract_mda_via_toc(soup)
    if mda_text:
        return mda_text
    
    # Strategy 2: Look for MD&A by section headings
    mda_text = extract_mda_via_headings(soup)
    if mda_text:
        return mda_text
    
    # Strategy 3: Look for MD&A by common patterns
    mda_text = extract_mda_via_patterns(soup)
    if mda_text:
        return mda_text
    
    return None

def extract_mda_via_toc(soup):
    """Extract MD&A using table of contents links."""
    # Regex patterns for start and end
    pattern_start = re.compile(
        r"management[’'s]*\s*discussion\s*and\s*analysis", 
        re.IGNORECASE
    )
    
    pattern_end = re.compile(
        r"(quantitative.*qualitative|risk\s*factors|financial\s*statements|notes\s*to\s*financial)",
        re.IGNORECASE
    )
    
    start_href = None
    end_href = None
    
    # Find start and end hrefs
    for a in soup.find_all("a"):
        text = a.get_text(" ", strip=True)
        if pattern_start.search(text):
            start_href = a.get("href")
            break
    
    if not start_href:
        return None
    
    # Look for end marker
    for a in soup.find_all("a"):
        text = a.get_text(" ", strip=True)
        if pattern_end.search(text):
            end_href = a.get("href")
            break
    
    # Function to find anchor target
    def find_anchor_target(href):
        if not href:
            return None
        
        if href.startswith("#"):
            anchor_name = href[1:]
            
            # Try ID first
            t = soup.find(id=anchor_name)
            if t:
                return t
            
            # Try <a name="">
            t = soup.find("a", attrs={"name": anchor_name})
            if t:
                return t
            
            # Try any tag with name=""
            t = soup.find(attrs={"name": anchor_name})
            if t:
                return t
        
        return None
    
    start_anchor = find_anchor_target(start_href)
    end_anchor = find_anchor_target(end_href) if end_href else None
    
    # Extract text
    return extract_text_between_anchors(soup, start_anchor, end_anchor)

def extract_mda_via_headings(soup):
    """Extract MD&A by looking for section headings."""
    # Common MD&A headings
    mda_patterns = [
        re.compile(r"ITEM\s*[78]\.?\s*MANAGEMENT['S]*\s*DISCUSSION\s*AND\s*ANALYSIS", re.IGNORECASE),
        re.compile(r"MANAGEMENT['S]*\s*DISCUSSION\s*AND\s*ANALYSIS", re.IGNORECASE),
        re.compile(r"MD&A", re.IGNORECASE),
    ]
    
    # Common end patterns
    end_patterns = [
        re.compile(r"QUANTITATIVE\s*AND\s*QUALITATIVE", re.IGNORECASE),
        re.compile(r"ITEM\s*[78][A-Z]*\.?", re.IGNORECASE),
        re.compile(r"RISK\s*FACTORS", re.IGNORECASE),
        re.compile(r"FINANCIAL\s*STATEMENTS", re.IGNORECASE),
    ]
    
    # Look for MD&A heading
    start_element = None
    for pattern in mda_patterns:
        for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'div', 'p', 'b', 'strong']):
            text = tag.get_text(strip=True)
            if pattern.search(text):
                start_element = tag
                break
        if start_element:
            break
    
    if not start_element:
        return None
    
    # Look for end marker
    end_element = None
    for pattern in end_patterns:
        for sibling in start_element.find_next_siblings():
            text = sibling.get_text(strip=True)
            if pattern.search(text) and len(text) < 100:  # Likely a heading
                end_element = sibling
                break
        if end_element:
            break
    
    return extract_text_between_elements(soup, start_element, end_element)

def extract_mda_via_patterns(soup):
    """Extract MD&A by looking for patterns in text."""
    # Get all text
    all_text = soup.get_text()
    
    # Find MD&A section
    mda_pattern = re.compile(
        r'(ITEM\s*[78]\.?\s*)?MANAGEMENT[’\'\"]?S\s*DISCUSSION\s*AND\s*ANALYSIS.*?(?=ITEM\s*[78][A-Z]*|QUANTITATIVE\s*AND\s*QUALITATIVE|RISK\s*FACTORS|FINANCIAL\s*STATEMENTS|$)',
        re.IGNORECASE | re.DOTALL
    )
    
    match = mda_pattern.search(all_text)
    if match:
        # Clean up the extracted text
        mda_text = match.group(0)
        # Remove excessive whitespace
        mda_text = re.sub(r'\s+', ' ', mda_text)
        return mda_text.strip()
    
    return None

def extract_text_between_anchors(soup, start_anchor, end_anchor):
    """Extract text between two anchors."""
    if not start_anchor:
        return None
    
    texts = []
    
    for elem in start_anchor.next_elements:
        if elem == end_anchor:
            break
        
        if isinstance(elem, NavigableString):
            t = str(elem).strip()
            if t and len(t) > 5:  # Skip very short strings
                texts.append(t)
    
    if not texts:
        return None
    
    extracted_text = " ".join(texts)
    # Clean up the text
    extracted_text = re.sub(r'\s+', ' ', extracted_text)
    return extracted_text[:10000]  # Limit to first 10k characters

def extract_text_between_elements(soup, start_element, end_element):
    """Extract text between two elements."""
    texts = []
    
    current = start_element.next_sibling
    while current and current != end_element:
        if isinstance(current, NavigableString):
            t = str(current).strip()
            if t and len(t) > 5:
                texts.append(t)
        elif current.name in ['p', 'div', 'span']:
            text = current.get_text(strip=True)
            if text and len(text) > 10:
                texts.append(text)
        
        if hasattr(current, 'next_sibling'):
            current = current.next_sibling
        else:
            break
    
    if not texts:
        return None
    
    extracted_text = " ".join(texts)
    extracted_text = re.sub(r'\s+', ' ', extracted_text)
    return extracted_text[:10000]

# Function to get HTML file path
def get_html_file_path(ticker, year_qtr, base_dir="sec_filings"):
    """Construct the path to the HTML file."""
    # Try different possible file locations
    possible_paths = [
        os.path.join(base_dir, ticker, f"{year_qtr}.html"),
        os.path.join(base_dir, ticker, f"{year_qtr.lower()}.html"),
        os.path.join(base_dir, ticker, year_qtr, "filing.html"),
        os.path.join(base_dir, ticker, year_qtr.replace("_", ""), "filing.html"),
    ]
    
    for path in possible_paths:
        if os.path.exists(path):
            return path
    
    return None

# Main extraction with better logging
print("Extracting MD&A from HTML files...\n")

mda_list = []
success_count = 0
failed_count = 0

for idx, row in df.iterrows():
    ticker = row['Ticker']
    year_qtr = row['year_qtr']
    
    html_path = get_html_file_path(ticker, year_qtr)
    
    if html_path and os.path.exists(html_path):
        print(f"Processing {idx}: {ticker} - {year_qtr}")
        print(f"  File: {html_path}")
        
        mda_text = extract_mda_from_html(html_path, verbose=True)
        
        if mda_text:
            # Truncate and show preview
            preview = mda_text[:200] + "..." if len(mda_text) > 200 else mda_text
            print(f"  SUCCESS: Extracted {len(mda_text)} characters")
            print(f"  Preview: {preview}")
            mda_list.append(mda_text)
            success_count += 1
        else:
            print(f"  FAILED: Could not extract MD&A")
            mda_list.append(None)
            failed_count += 1
    else:
        print(f"Skipping {idx}: File not found - {ticker}/{year_qtr}")
        mda_list.append(None)
        failed_count += 1
    
    print()

# Add MD&A column to dataframe
df['md&a'] = mda_list

print(f"Extraction complete!")
print(f"Success: {success_count}, Failed/Not Found: {failed_count}")
print(f"Success rate: {success_count/len(df)*100:.1f}%")

# Save the enhanced dataframe
df.to_csv('02_credit_ratings_with_mda.csv', index=False)
print(f"\\nSaved enhanced data to '02_credit_ratings_with_mda.csv'")

# Show statistics
print(f"\\nRows with MD&A data: {df['md&a'].notna().sum()}")
print(f"Rows without MD&A data: {df['md&a'].isna().sum()}")

# Preview rows with MD&A
if df['md&a'].notna().any():
    print("\\nSample of extracted MD&A data:")
    preview_df = df[df['md&a'].notna()][['Ticker', 'year_qtr', 'Rating']].head()
    for _, row in preview_df.iterrows():
        print(f"  {row['Ticker']} ({row['year_qtr']}) - Rating: {row['Rating']}")

Extracting MD&A from HTML files...

Processing 0: WHR - 2015_Q4
  File: sec_filings\WHR\2015_Q4.html
  SUCCESS: Extracted 10000 characters
  Preview: ABOUT WHIRLPOOL Whirlpool is the number one major appliance manufacturer in the world with net sales of approximately $20 billion and net earnings available to Whirlpool of $650 million . We are a lea...

Processing 1: WHR - 2014_Q1
  File: sec_filings\WHR\2014_Q1.html
  SUCCESS: Extracted 10000 characters
  Preview: ABOUT WHIRLPOOL Whirlpool Corporation (“Whirlpool”) is the world’s leading manufacturer of major home appliances with revenues of approximately $18 billion and net earnings available to Whirlpool of $...

Processing 2: WHR - 2015_Q1
  File: sec_filings\WHR\2015_Q1.html
  SUCCESS: Extracted 10000 characters
  Preview: ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS This Management Discussion and Analysis should be read in connection with the Consolidated Financial State...

Process